In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import svm
import argparse
import cv2
import matplotlib.pyplot as plt

import os

paths=[]
labels=[]

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths+=[os.path.join(dirname, filename)]
        labels+=[dirname.split('/')[-1]]

print(paths[0:3])

['/kaggle/input/eye-diseases-classification/dataset/glaucoma/1255_right.jpg', '/kaggle/input/eye-diseases-classification/dataset/glaucoma/1482_left.jpg', '/kaggle/input/eye-diseases-classification/dataset/glaucoma/_398_8823411.jpg']


In [2]:
def get_lines(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR) # road.png is the filename
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 200)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 10, minLineLength=10, maxLineGap=250)
#     lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, None, 50, 10)
#     lines = cv2.HoughLines(edges, 1, np.pi / 180, 150, 100, 0, 0)
    
    return lines

In [3]:
def get_circles(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.medianBlur(gray, 5)
    circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, 40, param1=100, param2=30, minRadius=1, maxRadius=40)

    return circles

In [4]:
def extract_feature_hough(img_paths_df):
#     print((img_paths_df))
    data = []
    labels = []
    counter = 1
    for i in img_paths_df.index:
        img_path = img_paths_df["path"][i]
        label = img_paths_df["label"][i]
        if counter % 50 == 0:
            print(counter)
#         image = cv2.imread(img_path)
# #         print(img_path)
# #         plt.imshow(plt.imread(img_path))
# #         plt.show()
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         hist = desc.describe(gray)
        
#         img_blur = cv2.medianBlur(gray, 5)
#         circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT,1, 40, param1=100, param2=30, minRadius=1, maxRadius=40)
#         circles = circles if circles is not None else np.zeros((1,1,1))
#         cycle_count = circles.flatten()
        
#         edges = cv2.Canny(gray, 50, 200)
#         lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=10,maxLineGap=250)
        
        lines = get_lines(img_path)
        circles = get_circles(img_path)
        circles = circles if circles is not None else np.zeros((1,1,1))
        lines = lines if lines is not None else np.zeros((1,1,1,1))
        
        shite = []
        for line in lines:
            shite.extend(line[0])
    
        labels.append(label)
#         data.append([len(lines), len(circles)])
        data.append(shite)
#         data.append
        
        counter += 1
    
    return labels, data


In [5]:
len(np.zeros((1,1,1, 1)))
a = [1,2,3,4]
b = [10]
b.extend(a)
print(b)


[10, 1, 2, 3, 4]


In [6]:
data_df=pd.DataFrame(columns=['path','label'])
data_df['path']=paths
data_df['label']=labels
data_df = data_df.sample(frac=1).reset_index(drop=True)

# display(data_df)
# print(len(data_df))
display(data_df['label'].value_counts())
# data_df.head()

diabetic_retinopathy    1098
normal                  1074
cataract                1038
glaucoma                1007
Name: label, dtype: int64

In [7]:
data_df = data_df[:10]
print(len(data_df))
train_data_df, test_data_df = train_test_split(data_df, test_size=0.2)

10


In [8]:
for i in train_data_df.index:
    img_path = train_data_df["path"][i]
    label = train_data_df["label"][i]

    lines = get_lines(img_path)
    circles = get_circles(img_path)
    
    if lines is not None:
        for i in range(0, len(lines)):
            l = lines[i][0]
            print(l)
            break
    
#     print(lines)
#     shite = []
#     for line in lines:
# #         print(line[0])
#         shite.extend(line[0])
# #         break
#     print(shite)
    break
    



[275  22 400 459]


In [9]:
img_labels, img_data = extract_feature_hough(train_data_df)

In [10]:
# for shit in img_data:
#     print(shit)
print(len(img_data))
print(img_labels)

8
['diabetic_retinopathy', 'normal', 'diabetic_retinopathy', 'diabetic_retinopathy', 'glaucoma', 'glaucoma', 'glaucoma', 'cataract']


In [11]:
img_labels_test, img_data_test = extract_feature_hough(test_data_df)

In [12]:
model = LinearSVC(dual=False, C=100.0, random_state=42)
model.fit(img_data, img_labels)
pred = model.predict(img_data_test)

from sklearn import metrics
print("Feature Extraction using: (HOUGH)\nAccuracy:", round(metrics.accuracy_score(img_labels_test, y_pred=pred) * 100,3),"%")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.

In [ ]:
linear = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(img_data, img_labels)
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(img_data, img_labels)
poly = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo').fit(img_data, img_labels)
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(img_data, img_labels)

In [ ]:
linear_pred = linear.predict(img_data_test)
poly_pred = poly.predict(img_data_test)
rbf_pred = rbf.predict(img_data_test)
sig_pred = sig.predict(img_data_test)

In [ ]:
accuracy_lin = linear.score(img_data_test, img_labels_test)
accuracy_poly = poly.score(img_data_test, img_labels_test)
accuracy_rbf = rbf.score(img_data_test, img_labels_test)
accuracy_sig = sig.score(img_data_test, img_labels_test)

print("Accuracy Linear Kernel:", round(accuracy_lin * 100,3),"%")
print("Accuracy Polynomial Kernel:", round(accuracy_poly * 100,3),"%")
print("Accuracy Radial Basis Kernel:", round(accuracy_rbf * 100,3),"%")
print("Accuracy Sigmoid Kernel:", round(accuracy_sig * 100,3),"%")